# NYC Airbnb Price Prediction - Data Preparation

Use dataset published by Kaggle - https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data - to train a simple deep learning model to predict prices for Airbnb properties.


This notebook contains the common data loading and preparation steps:
- load data from the input CSV
- fix missing values
- clean up anomalies

# Common imports and variables
Imports and variable definitions that are common to the entire notebook


In [153]:
!pip install requests
!pip install xlrd

You are using pip version 19.0.3, however version 20.2b1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You are using pip version 19.0.3, however version 20.2b1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [154]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import datetime as dt
# common imports
import zipfile
import time
# import datetime, timedelta
import datetime
from datetime import datetime, timedelta
from datetime import date
from dateutil import relativedelta
from io import StringIO
import pandas as pd
import pickle
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
from io import StringIO
import requests
import json
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline 
import os
import math
from subprocess import check_output
from IPython.display import display
import logging
import yaml
from collections import Counter
import re
import os


In [155]:
# load config file
current_path = os.getcwd()
print("current directory is: "+current_path)

path_to_yaml = os.path.join(current_path, 'airbnb_data_preparation_config.yml')
print("path_to_yaml "+path_to_yaml)
try:
    with open (path_to_yaml, 'r') as c_file:
        config = yaml.safe_load(c_file)
except Exception as e:
    print('Error reading the config file')
    

current directory is: C:\personal\manning\second_example_june_2020\airbnb\dl_example_2\notebooks
path_to_yaml C:\personal\manning\second_example_june_2020\airbnb\dl_example_2\notebooks\airbnb_data_preparation_config.yml


In [156]:
# common variables
# control whether to load data from scratch from original source or from saved dataframe
load_from_scratch = config['general']['load_from_scratch']
# control whether to save dataframe with transformed data
save_transformed_dataframe = config['general']['save_transformed_dataframe']
# control whether rows containing erroneous values are removed from the saved dataset
remove_bad_values = config['general']['remove_bad_values']
# load default replacements for missing values
text_default = config['general']['text_default']
categorical_default = config['general']['categorical_default']
time_default = config['general']['time_default']
continuous_default = config['general']['continuous_default']
# original CSV version of input (unprocessed) dataset
input_csv = config['file_names']['input_csv']
# saved pickled version of input dataset
pickled_input_dataframe = config['file_names']['pickled_input_dataframe']
# name of file to which prepared data set is saved as a pickled dataframe
pickled_output_dataframe = config['file_names']['pickled_output_dataframe']
# load lists of column categories
collist = config['categorical']
textcols = config['text']
continuouscols = config['continuous']
excludefromcolist = config['excluded']


In [157]:
print("load_from_scratch "+str(load_from_scratch))
print("save_transformed_dataframe "+str(save_transformed_dataframe))
print("remove_bad_values "+str(remove_bad_values))
print("pickled_input_dataframe "+str(pickled_input_dataframe))
print("pickled_output_dataframe "+str(pickled_output_dataframe))
print("defaults for text categorical time continuous are "+text_default+", "+categorical_default+", "+str(time_default)+", "+str(continuous_default))
print("collist is: ",str(collist))
print("textcols is: ",str(textcols))
print("continuouscols is: ",str(continuouscols))
print("excludefromcolist is: ",str(excludefromcolist))

load_from_scratch True
save_transformed_dataframe True
remove_bad_values True
pickled_input_dataframe AB_NYC_2019_df.pkl
pickled_output_dataframe AB_NYC_2019_remove_bad_values_jun21_2020.pkl
defaults for text categorical time continuous are missing, missing, 2019-01-01, 0.0
collist is:  ['neighbourhood_group', 'neighbourhood', 'room_type']
textcols is:  ['name', 'host_name']
continuouscols is:  ['minimum_nights', 'number_of_reviews', 'reviews_per_month', 'calculated_host_listings_count']
excludefromcolist is:  ['price', 'id', 'latitude', 'longitude', 'name', 'host_name', 'last_review']


# Load Data
- ingest CVS into a Pandas dataframe 

In [ ]:
# get the directory for that this notebook is in and return the directory containing data files

def get_path():
    rawpath = os.getcwd()
    # data is in a directory called "data" that is a sibling to the directory containing the notebook
    path = os.path.abspath(os.path.join(rawpath, '..', 'data'))
    return(path)


In [ ]:
# given a path return the list of xls files in the directory
def get_xls_list(path):
    files = os.listdir(path)
    files_xls = [f for f in files if f[-4:] == 'xlsx']
    print(files)
    print(files_xls)
    return(files_xls)


In [ ]:
# define categories for input columns
def define_feature_categories(df):
    allcols = list(df)
    print("all cols",allcols)
    textcols = ['name','host_name'] # 
    continuouscols = ['price','minimum_nights','number_of_reviews','reviews_per_month','calculated_host_listings_count','availability_365'] 
                      # columns to deal with as continuous values - no embeddings
    timecols = ['last_review']
    collist = ['neighbourhood_group','neighbourhood','room_type']
    for col in continuouscols:
        df[col] = df[col].astype(float)
    print('texcols: ',textcols)
    print('continuouscols: ',continuouscols)
    print('timecols: ',timecols)
    print('collist: ',collist)
    return(allcols,textcols,continuouscols,timecols,collist)

In [ ]:
# fill missing values according to the column category
def fill_missing(dataset,allcols,textcols,continuouscols,timecols,collist):
    logging.debug("before mv")
    for col in collist:
        dataset[col].fillna(value=categorical_default, inplace=True)
    for col in continuouscols:
        dataset[col].fillna(value=continuous_default,inplace=True)
    for col in timecols:
        dataset[col].fillna(value=time_default,inplace=True)
    for col in textcols:
        dataset[col].fillna(value=text_default, inplace=True)
    return (dataset)

# Load dataframe
- load pickled dataframe
- show info about the dataset


In [ ]:
# read in data, either from original CSV file in data directory or from saved pickled dataframe
def ingest_data(path):
    if load_from_scratch:
        unpickled_df = pd.read_csv(os.path.join(path,input_csv)) 
    else:
        unpickled_df = pd.read_pickle(os.path.join(path,pickled_input_dataframe))
        logging.debug("reloader done")
    return(unpickled_df)

# General cleanup
- correct types for Route and Vehicle
- fill missing values
- create report-date-time index

In [ ]:
# the dataset incorporated some anomalies in the 2019 data, including:
# extraneous Incident ID in April 2019 tab
# Gap and Delay columns in April and June 2019 tabs for what had otherwise been called Min Gap and Min Delay
# this function cleans up these anomalies
def fix_anomalous_columns(df):
    # for rows where there is NaN in the Min Delay or Min Gap columns, copy over value from Delay or Gap
    # df.Temp_Rating.fillna(df.Farheit, inplace=True)
    df['Min Delay'].fillna(df['Delay'], inplace=True)
    df['Min Gap'].fillna(df['Gap'], inplace=True)
    # now that the useful values have been copied from Delay and Gap, remove them
    del df['Delay']
    del df['Gap']
    # remove Incident ID column - it's extraneous
    del df['Incident ID']
    return(df)

In [ ]:
def replace_time(date_time_value,time_value):
    ''' given a datetime replace the time portion '''
     
    date_time_value = date_time_value.replace(hour=time_value.hour,minute=time_value.minute,second=time_value.minute)
    return(date_time_value)


In [ ]:
def general_cleanup(df):
    # ensure Route and Vehicle are strings, not numeric
    df['Route'] = df['Route'].astype(str)
    df['Vehicle'] = df['Vehicle'].astype(str)
    # remove extraneous characters left from Vehicle values being floats
    df['Vehicle'] = df['Vehicle'].str[:-2]
    # tactical definition of categories
    allcols,textcols,continuouscols,timecols,collist = define_feature_categories(df)
    # fill in missing values
    df.isnull().sum(axis = 0)
    df = fix_anomalous_columns(df)
    df = fill_missing(df,allcols,textcols,continuouscols,timecols,collist)
    # create new column combining date + time (needed for resampling) and make it the index
    df['Report Date Time'] = df.apply(lambda x: replace_time(x['Report Date'], x['Time']), axis=1)
    df.index = df['Report Date Time']
    # return the updated dataframe along with the column category lists
    return(df,allcols,textcols,continuouscols,timecols,collist)

# Master cell
This cell contains calls to the other functions in this notebook to complete the data preparation

In [ ]:
# master cell to call the other functions
# get the path for data files
path = get_path()
print("path is ",path)
# load route direction and delay data datframes
df = ingest_data(path)
allcols,textcols,continuouscols,timecols,collist = define_feature_categories(df)
# iterate through columns to get basic information
for col in list(df):
    print("Missing values in ",col," ",str(df[col].isna().sum()))
    print("Distinct values ",str(df[col].nunique()))
df = fill_missing(df,allcols,textcols,continuouscols,timecols,collist)
df.head()

if save_transformed_dataframe:
    print("path is ",path)
    file_name = os.path.join(path,pickled_output_dataframe)
    print("file_name is ",file_name)
    df.to_pickle(file_name)
df.head()

In [ ]:
df.head()